In [2]:
import json
import requests
import pickle
import crashsimilarity.downloader as cd
from crashsimilarity.utils import StackTraceProcessor
import crashsimilarity.tmp as tmp
from gensim.models import doc2vec
import numpy as np
import matplotlib.pyplot as plt
import logging

In [ ]:
r = requests.get('https://crash-stats.mozilla.com/api/SuperSearch/?product=Firefox&_facets=signature&_facets_size=20')

In [ ]:
facets = r.json()['facets']
# facets

In [ ]:
sig1 = facets['signature'][6]
sig1

In [ ]:
sig2 = facets['signature'][10]
sig2

In [ ]:
traces = cd.SocorroDownloader().download_stack_traces_for_signature(sig1['term'], 150)
traces1 = list(traces)
traces = cd.SocorroDownloader().download_stack_traces_for_signature(sig2['term'], 150)
traces2 = list(traces)

In [ ]:
traces1 = [StackTraceProcessor.preprocess(i) for i in traces1]
traces2 = [StackTraceProcessor.preprocess(i) for i in traces2]

In [ ]:
vocab = pickle.load(open('data/compressed_vocab.pickle', 'rb'))
len(vocab)

In [ ]:
def compress(words):
    new = 0
    rv = []
    for w in words:
        if w not in vocab:
            new += 1
            rv.append(w)
        else:
            rv.append(str(vocab[w]))
    return new, rv

In [ ]:
compressed_traces1 = []
new = []
for t in traces1:
    n, c = compress(t)
    new.append(n)
    compressed_traces1.append(c)
sum(new)

In [ ]:
compressed_traces2 = []
new = []
for t in traces2:
    n, c = compress(t)
    new.append(n)
    compressed_traces2.append(c)
sum(new)

In [ ]:
sample = []
for i, c in enumerate(compressed_traces1):
    sample.append(doc2vec.TaggedDocument(c, [i]))
for i, c in enumerate(compressed_traces2):
    sample.append(doc2vec.TaggedDocument(c, [i+len(compressed_traces1)]))

In [ ]:
corpus = list(tmp.corpus_generator('data/new_clean_compressed.json'))

In [ ]:
model = doc2vec.Doc2Vec(size=200, window=15, iter=10, workers=8, min_count=1)
model.build_vocab(corpus)
len(model.wv.vocab)

In [ ]:
model.train(corpus)

In [ ]:
model.save('data/model/dm_d200_all.model')

In [ ]:
model = doc2vec.Doc2Vec.load('data/model/dm_d200_all.model')

In [ ]:
type(model)

In [ ]:
words = set()
for c in sample:
    for w in c.words:
        words.add(w)
words = list(words)
c = sum([1 if w in vocab.values() else 0 for w in words])
c

In [ ]:
type(words[0])

In [ ]:
sample[0]

In [ ]:
m = sum([1 if str(w) in model else 0 for w in words])
m

In [ ]:
len(vocab)

In [ ]:
sample[228]

In [ ]:
ans = tmp.rwmd_distances(model, sample, 228)

In [ ]:
logging.root.setLevel(logging.CRITICAL)
dist = np.zeros((len(sample), len(sample)), dtype=np.double)
for i in range(len(sample)):
    if i % 50 == 0:
        print(i)
    dist[i] = np.array(tmp.rwmd_distances(model, sample, i))[:,1]
logging.root.setLevel(logging.INFO)

In [ ]:
dist

In [ ]:
from sklearn.manifold import MDS
clf = MDS()
X_mds = clf.fit_transform(dist)
X_mds.shape

In [ ]:
color = 'r' * len(compressed_traces1) + 'b' * len(compressed_traces1)
plt.scatter(X_mds[:, 0], X_mds[:, 1], c=color)
plt.show()